### Практическое задание к уроку 9

Для обучения взял за основу не Bert, а Roberta. В ней примерно в два раза больше параметров. В качестве предобученной сети - siebert/sentiment-roberta-large-english.
Про датасет - пытался скачать данные с русскими твитами - архив data.zip не распаковывается, если по отдельности качать трейн и тест - не скачивается тест. Решил взять датасет с английскими твитами из предыдущих уроков, полагаю разница невелика, раз уж предобученная сеть сама разбирается с лемматизацией токенизацией и т.д.


до обучения на нашем датасете роберта показала точность 0.29, после обучения - 0.93.

In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split

In [45]:
from tqdm import tqdm

In [3]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.5 MB/s eta 0:00:00


In [5]:
from transformers import pipeline, RobertaTokenizer, RobertaModel, AutoTokenizer, AutoModelForSequenceClassification

In [6]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"denisegorov","key":"ec36751b6b2a254da4471aad1bef3be2"}'}

In [7]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download arkhoshghalb/twitter-sentiment-analysis-hatred-speech

  0% 0.00/1.89M [00:00<?, ?B/s]
100% 1.89M/1.89M [00:00<00:00, 141MB/s]


In [8]:
! mkdir dataset
! unzip twitter-sentiment-analysis-hatred-speech.zip -d dataset
! rm -R twitter-sentiment-analysis-hatred-speech.zip

Archive:  twitter-sentiment-analysis-hatred-speech.zip
  inflating: dataset/test.csv        
  inflating: dataset/train.csv       


In [9]:
df_train = pd.read_csv("/content/dataset/train.csv")
df_test = pd.read_csv("/content/dataset/test.csv")
df_train, df_val = train_test_split(df_train, test_size=0.3, shuffle=True, random_state=42)

In [10]:
df_train.head(3)

,id,label,tweet
9635,9636,0,summer timeð #summeriscoming #swimming #pic...
2447,2448,0,"dese niggas will show dese otha bitches on fb,..."
16134,16135,0,"""when you are you boost your immune system a..."


In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [12]:
df_val.shape

(9589, 3)

In [13]:
df_val.value_counts('label')

label
0    8905
1     684
dtype: int64

siebert/sentiment-roberta-large-english

In [16]:
model_path = "siebert/sentiment-roberta-large-english"

In [17]:
sentiment_analysis = pipeline("sentiment-analysis", model=model_path)
print(sentiment_analysis("I love this!"))

[{'label': 'POSITIVE', 'score': 0.9988656044006348}]


In [18]:
print(sentiment_analysis("i hate you"))

[{'label': 'NEGATIVE', 'score': 0.9991950392723083}]


In [19]:
idx = 42
print(df_train.iloc[idx]['tweet'])
print('label is', df_train.iloc[idx]['label'])
sentiment_analysis(df_train.iloc[idx]['tweet'])

happy friday!  #fashion #friday #happiness #fashionblogger #blogger #style #fashionista #friyay #styleblogger   
label is 0


[{'label': 'POSITIVE', 'score': 0.9989053010940552}]

In [20]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [21]:
example_text = 'Text example for sentiment analysis'
bert_input = tokenizer(example_text, padding='max_length', max_length=10,
                       truncation=True, return_tensors="pt")


print(bert_input['input_ids'])
print(bert_input['attention_mask'])

tensor([[    0, 39645,  1246,    13,  5702,  1966,     2,     1,     1,     1]])
tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0]])


In [22]:
tokenizer.convert_ids_to_tokens(1246), tokenizer.convert_ids_to_tokens(5702)

('Ġexample', 'Ġsentiment')

In [23]:
example_text = tokenizer.decode(bert_input.input_ids[0])
print(example_text)

<s>Text example for sentiment analysis</s><pad><pad><pad>


In [24]:
class TwitterDataset(torch.utils.data.Dataset):

    def __init__(self, txts, labels):
        self._labels = labels

        self.tokenizer = AutoTokenizer.from_pretrained("siebert/sentiment-roberta-large-english")
        self._txts = [self.tokenizer(text, padding='max_length', max_length=10,
                                     truncation=True, return_tensors="pt")
                      for text in txts]

    def __len__(self):
        return len(self._txts)

    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [25]:
y_train = df_train['label'].values
y_val = df_val['label'].values

train_dataset = TwitterDataset(df_train['tweet'], y_train)
valid_dataset = TwitterDataset(df_val['tweet'], y_val)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=64,
                          shuffle=True,
                          num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=1)

In [26]:
for txt, lbl in train_loader:
    print(txt.keys())
    print(txt['input_ids'].shape)
    break

dict_keys(['input_ids', 'attention_mask'])
torch.Size([64, 1, 10])


In [39]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):
        super().__init__()
        self.bert = RobertaModel.from_pretrained(model_path)
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(1024, 2)
        self.sigm = nn.Sigmoid()

    def forward(self, x, mask):

        _, pooled_output = self.bert(input_ids=x, attention_mask=mask, return_dict=False)

        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.sigm(linear_output)
        return final_layer

In [40]:
model = BertClassifier().to(device)
criterion = nn.CrossEntropyLoss()

optimizer = Adam(model.linear.parameters(), lr=0.001)

Some weights of RobertaModel were not initialized from the model checkpoint at siebert/sentiment-roberta-large-english and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
print(model)
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))
print("Parameters transfer learning:", sum([param.nelement() for param in model.linear.parameters()]))

BertClassifier(
  (bert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNo

In [44]:
model.eval()
total_loss_val, total_acc_val = 0.0, 0.0
for val_input, val_label in valid_loader:
    val_label = val_label.to(device)
    mask = val_input['attention_mask'].to(device)
    input_id = val_input['input_ids'].squeeze(1).to(device)

    output = model(input_id, mask)

    batch_loss = criterion(output, val_label)
    total_loss_val += batch_loss.item()

    acc = (output.argmax(dim=1) == val_label).sum().item()
    total_acc_val += acc
print(f'Val Loss Before Training: {total_loss_val / len(valid_dataset): .3f} \
    | Val Accuracy Before Training: {total_acc_val / len(valid_dataset): .3f}')

Val Loss Before Training:  0.012     | Val Accuracy Before Training:  0.292


In [46]:
for epoch_num in range(2):
    total_acc_train = 0
    total_loss_train = 0

    model.train()
    for train_input, train_label in tqdm(train_loader):
        mask = train_input['attention_mask'].to(device)
        input_id = train_input['input_ids'].squeeze(1).to(device)
        train_label = train_label.to(device)

        output = model(input_id, mask)

        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()

        acc = (output.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

        model.zero_grad()
        batch_loss.backward()
        optimizer.step()

    model.eval()
    total_loss_val, total_acc_val = 0.0, 0.0
    for val_input, val_label in valid_loader:
        val_label = val_label.to(device)
        mask = val_input['attention_mask'].to(device)
        input_id = val_input['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask)

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()

        acc = (output.argmax(dim=1) == val_label).sum().item()
        total_acc_val += acc

    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train Accuracy: {total_acc_train / len(train_dataset): .3f} \
        | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
        | Val Accuracy: {total_acc_val / len(valid_dataset): .3f}')

100%|██████████| 350/350 [01:44<00:00,  3.36it/s]


Epochs: 1 | Train Loss:  0.006         | Train Accuracy:  0.918         | Val Loss:  0.006         | Val Accuracy:  0.929


100%|██████████| 350/350 [01:50<00:00,  3.17it/s]


Epochs: 2 | Train Loss:  0.006         | Train Accuracy:  0.930         | Val Loss:  0.006         | Val Accuracy:  0.929
